<a href="https://colab.research.google.com/github/skaumbdoallsaws-coder/AI-Drawing-Inspector/blob/main/ai_inspector_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Engineering Drawing Inspector v3.0

**Multi-Model QC Pipeline**

Outputs:
1. `ResolvedPartIdentity.json`
2. `QwenUnderstanding.json` (features, quality, BOM, mfg notes)
3. `DrawingEvidence.json`
4. `DiffResult.json`
5. `QCReport.md`

**v3.0 Enhancements (Planned):**
- Batch processing mode
- Low feature count warnings
- Improved deduplication
- GD&T symbol detection

In [ ]:
# Cell 1: Install Dependencies
!pip install -q pymupdf opencv-python-headless jsonschema pillow pytesseract
!pip install -q accelerate qwen-vl-utils bitsandbytes
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q json-repair openai
!apt-get install -y poppler-utils tesseract-ocr > /dev/null 2>&1
print("Dependencies installed!")

In [ ]:
# Cell 2: Imports and Configuration
import os, json, re, gc
import torch
import fitz
import numpy as np
from PIL import Image
from pathlib import Path
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime

# Configuration
DRAWING_PDF_PATH = ""
SOLIDWORKS_JSON_DIR = "sw_json_library"
OUTPUT_DIR = "qc_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Cell 3: BOM-Robust JSON Loader
def load_json_robust(filepath) -> Tuple[Optional[Dict], Optional[str]]:
    """Load JSON with BOM handling. Tries: utf-8-sig, utf-8, latin-1"""
    filepath = Path(filepath)
    for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            with open(filepath, 'r', encoding=enc) as f:
                return json.load(f), None
        except UnicodeDecodeError:
            continue
        except json.JSONDecodeError as e:
            if 'BOM' in str(e) and enc == 'utf-8':
                continue
            return None, f"JSON error: {str(e)[:50]}"
        except Exception as e:
            return None, f"Error: {str(e)[:50]}"
    return None, "Failed all encodings"

print("load_json_robust defined")

In [ ]:
# Cell 4: PDF Rendering
@dataclass
class PageArtifact:
    pageIndex0: int
    page: int
    image: Image.Image
    width: int
    height: int
    dpi: int
    direct_text: Optional[str] = None

def render_pdf(pdf_path: str, dpi: int = 300) -> List[PageArtifact]:
    """Render first page of PDF to image."""
    artifacts = []
    doc = fitz.open(pdf_path)
    page = doc.load_page(0)
    zoom = dpi / 72.0
    pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    direct_text = page.get_text("text")

    artifacts.append(PageArtifact(
        pageIndex0=0, page=1, image=img,
        width=pix.width, height=pix.height, dpi=dpi,
        direct_text=direct_text if len(direct_text.strip()) > 10 else None
    ))
    doc.close()
    print(f"Rendered: {pix.width}x{pix.height}px")
    return artifacts

print("render_pdf defined")

In [ ]:
# Cell 5: SolidWorks JSON Library
@dataclass
class SwPartEntry:
    json_path: str
    part_number: str
    filename_stem: str = ""
    data: Dict[str, Any] = field(default_factory=dict)

class SwJsonLibrary:
    def __init__(self):
        self.by_part_number: Dict[str, SwPartEntry] = {}
        self.by_filename: Dict[str, SwPartEntry] = {}
        self.all_entries: List[SwPartEntry] = []

    def _normalize(self, s: str) -> str:
        return re.sub(r'[-\s_]', '', str(s or '')).lower()

    def load_from_directory(self, directory: str):
        json_files = list(Path(directory).glob("**/*.json"))
        print(f"Found {len(json_files)} JSON files")

        for jp in json_files:
            data, err = load_json_robust(jp)
            if data is None:
                continue
            pn = data.get('identity', {}).get('partNumber', '')
            entry = SwPartEntry(str(jp), pn, jp.stem, data)
            self.all_entries.append(entry)
            if pn:
                self.by_part_number[pn] = entry
                self.by_part_number[self._normalize(pn)] = entry
            self.by_filename[jp.stem] = entry
            self.by_filename[self._normalize(jp.stem)] = entry
        print(f"Loaded {len(self.all_entries)} files")

    def lookup(self, candidate: str) -> Optional[SwPartEntry]:
        if not candidate:
            return None
        norm = self._normalize(candidate)
        return self.by_part_number.get(candidate) or self.by_part_number.get(norm) or \
               self.by_filename.get(candidate) or self.by_filename.get(norm)

sw_library = SwJsonLibrary()
print("SwJsonLibrary defined")

In [ ]:
# Cell 6: Part Identity Resolution (Robust Matching)

@dataclass
class ResolvedPartIdentity:
    partNumber: str
    confidence: float
    source: str
    swJsonPath: Optional[str] = None
    candidates_tried: List[str] = field(default_factory=list)

def clean_filename(filename: str) -> str:
    """Remove known suffixes like Paint, REV, etc."""
    cleaned = re.sub(r'[\s_]*(Paint|PAINT)$', '', filename, flags=re.IGNORECASE)
    return cleaned.strip()

def extract_pn_candidates(filename: str) -> List[str]:
    """
    Extract potential part number candidates from filename.
    Handles: 1013572_01, 101357201-03, 314884W_0, 046-935-REV-A
    Returns list of candidates (most specific to least).
    """
    name_no_ext = os.path.splitext(filename)[0]
    # Remove duplicate markers like (1), (2)
    name_no_ext = re.sub(r'\s*\(\d+\)$', '', name_no_ext)
    cleaned = clean_filename(name_no_ext)
    parts = re.split(r'[\s_]+', cleaned)

    if not parts:
        return []

    base = parts[0]
    candidates = []

    # 1. Base as-is
    candidates.append(base)

    # 2. Without hyphens
    base_no_hyphen = base.replace('-', '')
    if base_no_hyphen != base:
        candidates.append(base_no_hyphen)

    # 3. Remove letter suffixes (046-935A -> 046-935)
    if base and base[-1].isalpha() and len(base) > 1:
        candidates.append(base[:-1])
        candidates.append(base[:-1].replace('-', ''))

    # 4. Handle revision pattern (046-935-01 -> 046-935)
    rev_match = re.match(r'^(.+)-(\d{1,2})$', base)
    if rev_match:
        main_part = rev_match.group(1)
        candidates.append(main_part)
        candidates.append(main_part.replace('-', ''))

    # 5. Handle REV suffix (046-935-REV-A -> 046-935)
    rev_alpha = re.match(r'^(.+?)[-_]?REV[-_]?[A-Z0-9]*$', base, re.IGNORECASE)
    if rev_alpha:
        candidates.append(rev_alpha.group(1))
        candidates.append(rev_alpha.group(1).replace('-', ''))

    # 6. Peeling - progressively remove trailing digits
    temp = base_no_hyphen
    while len(temp) > 5:
        temp = temp[:-1]
        candidates.append(temp)

    # Remove duplicates, preserve order
    seen = set()
    unique = []
    for c in candidates:
        if c and c not in seen:
            seen.add(c)
            unique.append(c)

    return unique

def resolve_part_identity(pdf_path: str, artifacts: List[PageArtifact], sw_lib: SwJsonLibrary) -> ResolvedPartIdentity:
    """Resolve part identity using robust filename matching."""
    filename = os.path.basename(pdf_path)
    candidates = extract_pn_candidates(filename)

    # Try each candidate against SW library
    for candidate in candidates:
        entry = sw_lib.lookup(candidate)
        if entry:
            return ResolvedPartIdentity(
                partNumber=entry.part_number or candidate,
                confidence=1.0,
                source="filename+sw",
                swJsonPath=entry.json_path,
                candidates_tried=candidates
            )

    # Try PDF embedded text
    for art in artifacts:
        if art.direct_text:
            text_candidates = extract_pn_candidates(art.direct_text[:200])
            for candidate in text_candidates[:5]:
                entry = sw_lib.lookup(candidate)
                if entry:
                    return ResolvedPartIdentity(
                        partNumber=entry.part_number or candidate,
                        confidence=0.8,
                        source="pdf_text+sw",
                        swJsonPath=entry.json_path,
                        candidates_tried=candidates + text_candidates[:5]
                    )

    # Fallback - use first candidate or filename stem
    fallback_pn = candidates[0] if candidates else Path(pdf_path).stem
    return ResolvedPartIdentity(
        partNumber=fallback_pn,
        confidence=0.3,
        source="fallback",
        swJsonPath=None,
        candidates_tried=candidates
    )

print("resolve_part_identity defined (robust matching)")

In [ ]:
# Cell 7: Load SolidWorks Library (Upload ZIP)
from google.colab import files
import zipfile

if not os.path.exists(SOLIDWORKS_JSON_DIR) or not list(Path(SOLIDWORKS_JSON_DIR).glob("*.json")):
    print("Upload your sw_json_library.zip file:")
    uploaded = files.upload()

    for filename in uploaded:
        if filename.endswith('.zip'):
            print(f"Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as z:
                z.extractall(SOLIDWORKS_JSON_DIR)
            print(f"Extracted to {SOLIDWORKS_JSON_DIR}")
            break

sw_library.load_from_directory(SOLIDWORKS_JSON_DIR)
print(f"Library ready: {len(sw_library.all_entries)} parts indexed")

In [ ]:
# Cell 8: Upload and Render PDF Drawing
from google.colab import files
from IPython.display import display

print("Upload your PDF drawing:")
uploaded = files.upload()

for filename in uploaded:
    if filename.lower().endswith('.pdf'):
        DRAWING_PDF_PATH = filename
        break

print(f"Processing: {DRAWING_PDF_PATH}")
artifacts = render_pdf(DRAWING_PDF_PATH)

# Display the rendered image
if artifacts:
    display(artifacts[0].image.resize((800, int(800 * artifacts[0].height / artifacts[0].width))))

In [ ]:
# Cell 9: Resolve Part Identity
part_identity = resolve_part_identity(DRAWING_PDF_PATH, artifacts, sw_library)

print("="*50)
print("RESOLVED PART IDENTITY")
print("="*50)
print(f"Part Number:  {part_identity.partNumber}")
print(f"Confidence:   {part_identity.confidence}")
print(f"Source:       {part_identity.source}")
print(f"SW JSON:      {part_identity.swJsonPath or 'Not found'}")
print(f"Candidates:   {part_identity.candidates_tried[:5]}")

# Save to output
identity_out = os.path.join(OUTPUT_DIR, "ResolvedPartIdentity.json")
with open(identity_out, 'w') as f:
    json.dump(asdict(part_identity), f, indent=2)
print(f"\nSaved: {identity_out}")

In [ ]:
# Cell 10: Load LightOnOCR-2 and Run OCR
from transformers import LightOnOcrForConditionalGeneration, LightOnOcrProcessor
from google.colab import userdata

# Clear GPU memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Get HF token
try:
    hf_token = userdata.get('HF_TOKEN')
except:
    hf_token = None

print("Loading LightOnOCR-2-1B...")
ocr_device = "cuda" if torch.cuda.is_available() else "cpu"
ocr_dtype = torch.bfloat16 if ocr_device == "cuda" else torch.float32

ocr_processor = LightOnOcrProcessor.from_pretrained(
    "lightonai/LightOnOCR-2-1B",
    token=hf_token
)

ocr_model = LightOnOcrForConditionalGeneration.from_pretrained(
    "lightonai/LightOnOCR-2-1B",
    torch_dtype=ocr_dtype,
    token=hf_token
).to(ocr_device)

print(f"LightOnOCR-2 loaded: {ocr_model.get_memory_footprint() / 1e9:.2f} GB")

def run_lighton_ocr(image: Image.Image) -> List[str]:
    """Run LightOnOCR-2 on image, return list of text lines."""
    global ocr_model, ocr_processor, ocr_device, ocr_dtype

    img = image.convert("RGB")
    conversation = [{"role": "user", "content": [{"type": "image", "image": img}]}]

    inputs = ocr_processor.apply_chat_template(
        conversation, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    )
    inputs = {k: v.to(device=ocr_device, dtype=ocr_dtype) if v.is_floating_point() else v.to(ocr_device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = ocr_model.generate(**inputs, max_new_tokens=2048)

    generated_ids = output_ids[0, inputs["input_ids"].shape[1]:]
    output_text = ocr_processor.decode(generated_ids, skip_special_tokens=True)

    return [line.strip() for line in output_text.split("\n") if line.strip()]

# Run OCR on the drawing
print("Running OCR on drawing...")
ocr_lines = run_lighton_ocr(artifacts[0].image)
print(f"OCR extracted {len(ocr_lines)} lines")
print("\nFirst 10 lines:")
for line in ocr_lines[:10]:
    print(f"  {line}")

In [ ]:
# Cell 10b: Qwen2.5-VL Drawing Understanding + Quality Audit + BOM + Mfg Notes
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from json_repair import repair_json

# Clear some GPU memory before loading Qwen
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Loading Qwen2.5-VL-7B for drawing understanding...")
qwen_model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

qwen_processor = AutoProcessor.from_pretrained(qwen_model_id, trust_remote_code=True)
qwen_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    qwen_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
print(f"Qwen2.5-VL loaded: {qwen_model.get_memory_footprint() / 1e9:.2f} GB")

def run_qwen_analysis(image: Image.Image, prompt: str) -> Dict[str, Any]:
    """Run Qwen2.5-VL with a given prompt and return parsed JSON."""
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    text = qwen_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(qwen_model.device)

    with torch.no_grad():
        output_ids = qwen_model.generate(**inputs, max_new_tokens=4096, temperature=0.1)

    generated_ids = output_ids[0, inputs.input_ids.shape[1]:]
    response = qwen_processor.decode(generated_ids, skip_special_tokens=True)

    # Parse JSON from response with repair
    try:
        json_match = re.search(r'```json\s*([\s\S]*?)\s*```', response)
        if json_match:
            json_str = json_match.group(1)
        else:
            json_match = re.search(r'\{[\s\S]*\}', response)
            json_str = json_match.group() if json_match else response

        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            print("  Attempting JSON repair...")
            repaired = repair_json(json_str)
            return json.loads(repaired)

    except Exception as e:
        return {"raw_response": response[:1000], "parse_error": str(e)}

# === ANALYSIS 1: Feature Extraction ===
feature_prompt = """Analyze this engineering drawing and identify all features. Return a JSON object with:

{
  "partDescription": "brief description of the part",
  "views": ["list of views shown: TOP, FRONT, SIDE, ISOMETRIC, SECTION, DETAIL"],
  "features": [
    {
      "type": "TappedHole|ThroughHole|BlindHole|Counterbore|Countersink|Slot|Fillet|Chamfer|Thread",
      "description": "brief description",
      "callout": "the dimension/callout text if visible",
      "quantity": 1,
      "location": "where on the part"
    }
  ],
  "material": "material if shown in title block",
  "titleBlockInfo": {
    "partNumber": "if visible",
    "revision": "if visible",
    "scale": "if visible"
  },
  "notes": ["any general notes visible on drawing"]
}

Be thorough - identify ALL holes, threads, chamfers, fillets, and other machined features you can see.
Only return valid JSON, no other text."""

print("Analyzing drawing features with Qwen2.5-VL...")
qwen_understanding = run_qwen_analysis(artifacts[0].image, feature_prompt)

print("="*50)
print("QWEN2.5-VL FEATURE ANALYSIS")
print("="*50)

if "parse_error" not in qwen_understanding:
    print(f"Part: {qwen_understanding.get('partDescription', 'N/A')}")
    print(f"Views: {qwen_understanding.get('views', [])}")
    print(f"Material: {qwen_understanding.get('material', 'N/A')}")
    print(f"\nFeatures identified: {len(qwen_understanding.get('features', []))}")
    for f in qwen_understanding.get('features', [])[:10]:
        print(f"  - {f.get('type')}: {f.get('callout', f.get('description', ''))}")
else:
    print(f"Parse error: {qwen_understanding.get('parse_error')}")

# === ANALYSIS 2: Drawing Quality Audit ===
quality_prompt = """Examine this engineering drawing for completeness and best practices. Return a JSON object with:

{
  "titleBlockCompleteness": {
    "hasPartNumber": true/false,
    "partNumberValue": "the part number if visible",
    "hasDescription": true/false,
    "descriptionValue": "the description if visible",
    "hasMaterial": true/false,
    "materialValue": "the material if visible",
    "hasRevision": true/false,
    "revisionValue": "the revision if visible",
    "hasScale": true/false,
    "scaleValue": "the scale if visible",
    "hasDate": true/false,
    "dateValue": "the date if visible",
    "hasDrawnBy": true/false,
    "drawnByValue": "name if visible",
    "hasApprovedBy": true/false,
    "approvedByValue": "name if visible"
  },
  "drawingQuality": {
    "viewsLabeled": true/false,
    "viewsLabeledComment": "are views clearly labeled (FRONT, TOP, etc)?",
    "dimensionsReadable": true/false,
    "dimensionsComment": "are dimensions clear and not overlapping?",
    "tolerancesPresent": true/false,
    "tolerancesComment": "are tolerances specified on critical dimensions?",
    "surfaceFinishSpecified": true/false,
    "surfaceFinishComment": "is surface finish callout present?",
    "generalToleranceBlock": true/false,
    "generalToleranceComment": "is there a general tolerance note/block?",
    "thirdAngleProjection": true/false,
    "projectionComment": "is projection symbol visible (third angle)?",
    "unitsSpecified": true/false,
    "unitsValue": "INCHES or MM if specified"
  },
  "overallAssessment": {
    "completenessScore": "1-10 rating",
    "majorIssues": ["list any major issues found"],
    "minorIssues": ["list any minor issues found"],
    "recommendations": ["suggestions for improvement"]
  }
}

Be thorough and critical - this is a quality audit. Only return valid JSON, no other text."""

print("\nRunning drawing quality audit with Qwen2.5-VL...")
drawing_quality = run_qwen_analysis(artifacts[0].image, quality_prompt)

print("\n" + "="*50)
print("QWEN2.5-VL DRAWING QUALITY AUDIT")
print("="*50)

if "parse_error" not in drawing_quality:
    tb = drawing_quality.get('titleBlockCompleteness', {})
    dq = drawing_quality.get('drawingQuality', {})
    oa = drawing_quality.get('overallAssessment', {})

    print("\nTitle Block:")
    print(f"  Part Number: {'✓' if tb.get('hasPartNumber') else '✗'} {tb.get('partNumberValue', '')}")
    print(f"  Description: {'✓' if tb.get('hasDescription') else '✗'} {tb.get('descriptionValue', '')[:30] if tb.get('descriptionValue') else ''}")
    print(f"  Material:    {'✓' if tb.get('hasMaterial') else '✗'} {tb.get('materialValue', '')}")
    print(f"  Revision:    {'✓' if tb.get('hasRevision') else '✗'} {tb.get('revisionValue', '')}")
    print(f"  Scale:       {'✓' if tb.get('hasScale') else '✗'} {tb.get('scaleValue', '')}")
    print(f"  Date:        {'✓' if tb.get('hasDate') else '✗'} {tb.get('dateValue', '')}")

    print("\nDrawing Quality:")
    print(f"  Views Labeled:     {'✓' if dq.get('viewsLabeled') else '✗'}")
    print(f"  Dims Readable:     {'✓' if dq.get('dimensionsReadable') else '✗'}")
    print(f"  Tolerances:        {'✓' if dq.get('tolerancesPresent') else '✗'}")
    print(f"  Surface Finish:    {'✓' if dq.get('surfaceFinishSpecified') else '✗'}")
    print(f"  General Tol Block: {'✓' if dq.get('generalToleranceBlock') else '✗'}")

    print(f"\nOverall Score: {oa.get('completenessScore', 'N/A')}/10")
else:
    print(f"Parse error: {drawing_quality.get('parse_error')}")
    drawing_quality = {}

# === ANALYSIS 3: BOM Extraction (for assembly drawings) ===
bom_prompt = """Look at this engineering drawing. If there is a Bill of Materials (BOM) or Parts List table visible, extract it. Return a JSON object:

{
  "hasBOM": true/false,
  "bomLocation": "where on the drawing (e.g., upper right, separate sheet)",
  "bomItems": [
    {
      "itemNumber": "1",
      "partNumber": "the part number",
      "description": "part description",
      "quantity": 1,
      "material": "if specified in BOM"
    }
  ],
  "totalItems": 0,
  "bomNotes": "any notes about the BOM (e.g., 'items not shown', 'see sheet 2')"
}

If there is NO BOM or Parts List visible, return:
{
  "hasBOM": false,
  "bomLocation": null,
  "bomItems": [],
  "totalItems": 0,
  "bomNotes": "No BOM found - this appears to be a detail/part drawing"
}

Only return valid JSON, no other text."""

print("\nExtracting BOM (if present) with Qwen2.5-VL...")
bom_data = run_qwen_analysis(artifacts[0].image, bom_prompt)

print("\n" + "="*50)
print("QWEN2.5-VL BOM EXTRACTION")
print("="*50)

if "parse_error" not in bom_data:
    if bom_data.get('hasBOM'):
        print(f"BOM Found: Yes ({bom_data.get('bomLocation', 'location unknown')})")
        print(f"Total Items: {bom_data.get('totalItems', len(bom_data.get('bomItems', [])))}")
        print("\nBOM Items:")
        for item in bom_data.get('bomItems', [])[:10]:
            print(f"  {item.get('itemNumber', '?')}. {item.get('partNumber', 'N/A')} - {item.get('description', 'N/A')} (Qty: {item.get('quantity', 1)})")
        if bom_data.get('bomNotes'):
            print(f"\nNotes: {bom_data.get('bomNotes')}")
    else:
        print("BOM Found: No")
        print(f"Note: {bom_data.get('bomNotes', 'This appears to be a part drawing without BOM')}")
else:
    print(f"Parse error: {bom_data.get('parse_error')}")
    bom_data = {"hasBOM": False, "bomItems": [], "bomNotes": "Parse error"}

# === ANALYSIS 4: Manufacturing Notes Extraction ===
mfg_prompt = """Examine this engineering drawing for manufacturing-related notes and specifications. Return a JSON object:

{
  "hasManufacturingNotes": true/false,
  "heatTreatment": {
    "specified": true/false,
    "specification": "e.g., HEAT TREAT TO 58-62 HRC",
    "hardness": "e.g., 58-62 HRC",
    "process": "e.g., carburize, through harden, case harden"
  },
  "surfaceFinish": {
    "specified": true/false,
    "generalFinish": "e.g., 125 RMS, 63 Ra",
    "specificFinishes": [
      {"surface": "bore", "finish": "32 Ra"},
      {"surface": "OD", "finish": "63 Ra"}
    ]
  },
  "platingOrCoating": {
    "specified": true/false,
    "type": "e.g., ZINC PLATE, ANODIZE, PAINT BLACK, POWDER COAT",
    "specification": "e.g., PER MIL-C-5541, CLASS 2",
    "thickness": "if specified"
  },
  "weldingNotes": {
    "specified": true/false,
    "weldSpec": "e.g., AWS D1.1",
    "weldTypes": ["fillet", "groove", "spot"],
    "notes": "any welding-specific notes"
  },
  "specialProcesses": [
    {
      "process": "e.g., stress relieve, shot peen, passivate",
      "specification": "details if given"
    }
  ],
  "inspectionRequirements": {
    "specified": true/false,
    "requirements": ["e.g., 100% INSPECT THREADS", "CMM REQUIRED", "FIRST ARTICLE"]
  },
  "generalNotes": [
    "REMOVE ALL BURRS AND SHARP EDGES",
    "BREAK EDGES .005-.015",
    "any other manufacturing notes"
  ],
  "certifications": ["e.g., MATERIAL CERT REQUIRED", "PPAP REQUIRED"]
}

Extract ALL manufacturing-related information visible on the drawing. If a category has no information, set specified to false.
Only return valid JSON, no other text."""

print("\nExtracting manufacturing notes with Qwen2.5-VL...")
mfg_notes = run_qwen_analysis(artifacts[0].image, mfg_prompt)

print("\n" + "="*50)
print("QWEN2.5-VL MANUFACTURING NOTES")
print("="*50)

if "parse_error" not in mfg_notes:
    ht = mfg_notes.get('heatTreatment', {})
    sf = mfg_notes.get('surfaceFinish', {})
    pc = mfg_notes.get('platingOrCoating', {})
    wn = mfg_notes.get('weldingNotes', {})

    print(f"\nHeat Treatment: {'✓ ' + ht.get('specification', '') if ht.get('specified') else '✗ Not specified'}")
    print(f"Surface Finish: {'✓ ' + sf.get('generalFinish', '') if sf.get('specified') else '✗ Not specified'}")
    print(f"Plating/Coating: {'✓ ' + pc.get('type', '') if pc.get('specified') else '✗ Not specified'}")
    print(f"Welding Notes: {'✓ ' + wn.get('weldSpec', '') if wn.get('specified') else '✗ Not specified'}")

    if mfg_notes.get('specialProcesses'):
        print("\nSpecial Processes:")
        for sp in mfg_notes.get('specialProcesses', []):
            print(f"  - {sp.get('process', 'Unknown')}")

    if mfg_notes.get('generalNotes'):
        print("\nGeneral Notes:")
        for note in mfg_notes.get('generalNotes', [])[:5]:
            print(f"  - {note}")

    if mfg_notes.get('inspectionRequirements', {}).get('specified'):
        print("\nInspection Requirements:")
        for req in mfg_notes.get('inspectionRequirements', {}).get('requirements', []):
            print(f"  - {req}")
else:
    print(f"Parse error: {mfg_notes.get('parse_error')}")
    mfg_notes = {"hasManufacturingNotes": False}

# Save all analyses
understanding_out = os.path.join(OUTPUT_DIR, "QwenUnderstanding.json")
with open(understanding_out, 'w') as f:
    json.dump({
        'featureAnalysis': qwen_understanding,
        'qualityAudit': drawing_quality,
        'bomExtraction': bom_data,
        'manufacturingNotes': mfg_notes
    }, f, indent=2)
print(f"\nSaved: {understanding_out}")

In [ ]:
# Cell 11: Merge OCR + Qwen Understanding into Enriched Evidence
# All dimensions in INCHES (matching drawing callouts)

# Regex patterns for extracting callouts from OCR
PATTERNS = {
    'metric_thread': r'M(\d+(?:\.\d+)?)\s*[xX]\s*(\d+(?:\.\d+)?)',
    'imperial_thread': r'(\d+/\d+)\s*-\s*(\d+)',
    'thru_hole': r'[oOØ∅φ]?\s*(\.?\d+\.?\d*|\d+)\s*(?:mm|MM|")?\s*THRU',
    'blind_hole': r'[oOØ∅φ]?\s*(\.?\d+\.?\d*|\d+)\s*[xX]\s*(\d+\.?\d*)\s*(?:DEEP|DP)',
    'fillet': r'\bR(\d+\.?\d*)\b',
    'chamfer': r'(\d+\.?\d*)\s*[xX]\s*45\s*[°]?',
}

def parse_ocr_callouts(ocr_lines: List[str]) -> List[Dict]:
    """Extract callouts from OCR text. Hole diameters stored in inches (as-is from drawing)."""
    callouts = []
    raw_text = "\n".join(ocr_lines)

    # Metric threads (M6x1.0) - keep in metric
    for match in re.finditer(PATTERNS['metric_thread'], raw_text, re.IGNORECASE):
        callouts.append({
            'calloutType': 'TappedHole',
            'thread': {'standard': 'Metric', 'nominalDiameterMm': float(match.group(1)), 'pitch': float(match.group(2))},
            'raw': match.group(0), 'source': 'ocr'
        })

    # Through holes - store diameter in inches (no conversion)
    for match in re.finditer(PATTERNS['thru_hole'], raw_text, re.IGNORECASE):
        raw = match.group(0)
        val = float(match.group(1))
        callouts.append({
            'calloutType': 'Hole',
            'diameterInches': val,  # Keep as inches
            'isThrough': True,
            'raw': raw,
            'source': 'ocr'
        })

    # Fillets - store in inches
    for match in re.finditer(PATTERNS['fillet'], raw_text, re.IGNORECASE):
        raw = match.group(0)
        val = float(match.group(1))
        callouts.append({
            'calloutType': 'Fillet',
            'radiusInches': val,
            'raw': raw,
            'source': 'ocr'
        })

    # Chamfers - store in inches
    for match in re.finditer(PATTERNS['chamfer'], raw_text, re.IGNORECASE):
        raw = match.group(0)
        val = float(match.group(1))
        callouts.append({
            'calloutType': 'Chamfer',
            'distance1Inches': val,
            'angleDegrees': 45,
            'raw': raw,
            'source': 'ocr'
        })

    return callouts

def parse_qwen_features(qwen_data: Dict) -> List[Dict]:
    """Convert Qwen features to callout format. Dimensions in inches."""
    callouts = []
    if "parse_error" in qwen_data:
        return callouts

    type_map = {
        'TappedHole': 'TappedHole', 'ThroughHole': 'Hole', 'BlindHole': 'Hole',
        'Counterbore': 'Hole', 'Countersink': 'Hole', 'Fillet': 'Fillet',
        'Chamfer': 'Chamfer', 'Thread': 'TappedHole', 'Slot': 'Slot'
    }

    # Filter out general notes misclassified as features
    note_keywords = ['REMOVE ALL BURRS', 'BREAK SHARP EDGES', 'DEBURR', 'CLEAN']

    for feat in qwen_data.get('features', []):
        ftype = feat.get('type', '')
        callout_type = type_map.get(ftype, ftype)
        callout = feat.get('callout', '')
        qty = feat.get('quantity', 1)

        # Skip if this looks like a general note, not a feature
        if any(kw in callout.upper() for kw in note_keywords):
            continue

        entry = {
            'calloutType': callout_type,
            'description': feat.get('description', ''),
            'location': feat.get('location', ''),
            'quantity': qty,
            'raw': callout,
            'source': 'qwen'
        }

        # Try to parse dimensions from Qwen's callout text
        if callout:
            # Metric threads
            thread_match = re.search(r'M(\d+(?:\.\d+)?)[xX](\d+(?:\.\d+)?)', callout)
            if thread_match:
                entry['thread'] = {
                    'standard': 'Metric',
                    'nominalDiameterMm': float(thread_match.group(1)),
                    'pitch': float(thread_match.group(2))
                }

            # Hole diameters - keep in inches
            hole_match = re.search(r'[oOØ∅φ]?\s*(\.?\d+\.?\d*)', callout)
            if hole_match and callout_type == 'Hole':
                val = float(hole_match.group(1))
                entry['diameterInches'] = val
                entry['isThrough'] = 'THRU' in callout.upper()

        callouts.append(entry)

    return callouts

def merge_evidence(ocr_callouts: List[Dict], qwen_callouts: List[Dict]) -> List[Dict]:
    """Merge OCR and Qwen callouts, preferring OCR for dimensions but using Qwen for context."""
    merged = []
    used_qwen = set()

    for ocr in ocr_callouts:
        merged_entry = ocr.copy()
        merged_entry['sources'] = ['ocr']

        # Try to find matching Qwen feature for additional context
        for qi, qwen in enumerate(qwen_callouts):
            if qi in used_qwen:
                continue
            if ocr.get('calloutType') == qwen.get('calloutType'):
                # Check if dimensions roughly match
                if ocr.get('thread') and qwen.get('thread'):
                    if ocr['thread'].get('nominalDiameterMm') == qwen['thread'].get('nominalDiameterMm'):
                        merged_entry['location'] = qwen.get('location', '')
                        merged_entry['description'] = qwen.get('description', '')
                        merged_entry['sources'].append('qwen')
                        used_qwen.add(qi)
                        break
                elif ocr.get('diameterInches') and qwen.get('diameterInches'):
                    # Compare in inches with 0.01" tolerance
                    if abs(ocr['diameterInches'] - qwen['diameterInches']) < 0.01:
                        merged_entry['location'] = qwen.get('location', '')
                        merged_entry['description'] = qwen.get('description', '')
                        merged_entry['sources'].append('qwen')
                        used_qwen.add(qi)
                        break

        merged.append(merged_entry)

    # Add Qwen features not matched to OCR (may be features OCR missed)
    for qi, qwen in enumerate(qwen_callouts):
        if qi not in used_qwen:
            qwen['sources'] = ['qwen_only']
            merged.append(qwen)

    return merged

# Parse both sources
ocr_callouts = parse_ocr_callouts(ocr_lines)
qwen_callouts = parse_qwen_features(qwen_understanding)

# Merge
merged_callouts = merge_evidence(ocr_callouts, qwen_callouts)

# Build enriched evidence
evidence = {
    'schemaVersion': '1.4.0',  # Updated: now uses inches
    'partNumber': part_identity.partNumber,
    'extractedAt': datetime.now().isoformat() + 'Z',
    'units': 'inches',  # Explicit unit declaration
    'sources': {
        'ocr': {'model': 'LightOnOCR-2-1B', 'lineCount': len(ocr_lines)},
        'vision': {'model': 'Qwen2.5-VL-7B', 'featureCount': len(qwen_callouts)}
    },
    'drawingInfo': {
        'views': qwen_understanding.get('views', []),
        'partDescription': qwen_understanding.get('partDescription', ''),
        'material': qwen_understanding.get('material', ''),
        'titleBlock': qwen_understanding.get('titleBlockInfo', {}),
        'notes': qwen_understanding.get('notes', [])
    },
    'foundCallouts': merged_callouts,
    'rawOcrSample': ocr_lines[:15]
}

print("="*50)
print("MERGED DRAWING EVIDENCE (inches)")
print("="*50)
print(f"OCR callouts:   {len(ocr_callouts)}")
print(f"Qwen features:  {len(qwen_callouts)}")
print(f"Merged total:   {len(merged_callouts)}")
print(f"\nDrawing info:")
print(f"  Views: {evidence['drawingInfo']['views']}")
print(f"  Material: {evidence['drawingInfo']['material']}")
print(f"\nMerged callouts:")
for c in merged_callouts[:10]:
    sources = '+'.join(c.get('sources', []))
    extra = f" [{sources}]"
    if c.get('location'):
        extra += f" @ {c['location']}"
    print(f"  {c['calloutType']}: {c.get('raw', c.get('description', ''))}{extra}")

# Save
evidence_out = os.path.join(OUTPUT_DIR, "DrawingEvidence.json")
with open(evidence_out, 'w') as f:
    json.dump(evidence, f, indent=2)
print(f"\nSaved: {evidence_out}")

In [ ]:
# Cell 12: Generate DiffResult (comparison in INCHES)

def extract_sw_requirements(sw_data: Dict) -> List[Dict]:
    """Extract requirements from SolidWorks JSON using comparison.holeGroups.
    Returns hole diameters in INCHES for direct comparison with drawing callouts."""
    requirements = []

    # Primary source: comparison.holeGroups (reconciled/canonical data)
    comparison = sw_data.get('comparison', {})
    hole_groups = comparison.get('holeGroups', [])

    for hg in hole_groups:
        hole_type = hg.get('holeType', '')
        canonical = hg.get('canonical', '')
        count = hg.get('count', 1)
        diameters = hg.get('diameters', {})
        thread = hg.get('thread', {})

        if hole_type == 'Tapped':
            # Tapped hole - thread info stays in metric (M6x1.0)
            requirements.append({
                'type': 'TappedHole',
                'thread': {
                    'standard': thread.get('standard', 'Metric'),
                    'nominalDiameterMm': thread.get('majorDiameterMm') or diameters.get('threadNominalDiameterMm'),
                    'pitch': thread.get('pitch'),
                    'callout': thread.get('callout', canonical)
                },
                'count': count,
                'canonical': canonical,
                'source': 'sw_comparison.holeGroups'
            })
        elif hole_type == 'Through':
            # Plain through hole - use INCHES for comparison
            diameter_inches = diameters.get('pilotOrTapDrillDiameterInches')
            requirements.append({
                'type': 'Hole',
                'diameterInches': diameter_inches,
                'isThrough': True,
                'count': count,
                'canonical': canonical,
                'canonicalInches': diameters.get('nearestStandardInch', ''),
                'source': 'sw_comparison.holeGroups'
            })
        elif hole_type == 'Blind':
            # Blind hole - use INCHES
            diameter_inches = diameters.get('pilotOrTapDrillDiameterInches')
            requirements.append({
                'type': 'Hole',
                'diameterInches': diameter_inches,
                'isThrough': False,
                'count': count,
                'canonical': canonical,
                'source': 'sw_comparison.holeGroups'
            })

    # Fallback: features.holeWizardHoles if no comparison data
    if not requirements:
        features = sw_data.get('features', {})
        for hole in features.get('holeWizardHoles', []):
            if hole.get('isTapped'):
                thread_size = hole.get('threadSize', '')
                m = re.match(r'M(\d+(?:\.\d+)?)[xX](\d+(?:\.\d+)?)', thread_size)
                if m:
                    requirements.append({
                        'type': 'TappedHole',
                        'thread': {
                            'standard': 'Metric',
                            'nominalDiameterMm': float(m.group(1)),
                            'pitch': float(m.group(2)),
                            'callout': thread_size
                        },
                        'count': hole.get('instanceCount', 1),
                        'source': 'sw_features.holeWizardHoles'
                    })
            else:
                # Convert meters to inches
                diameter_m = hole.get('diameter', 0)
                diameter_inches = diameter_m * 39.3701
                requirements.append({
                    'type': 'Hole',
                    'diameterInches': diameter_inches,
                    'isThrough': hole.get('isThrough', False),
                    'count': hole.get('instanceCount', 1),
                    'source': 'sw_features.holeWizardHoles'
                })

        # Fillets - convert to inches
        for fillet in features.get('fillets', []):
            radius_mm = fillet.get('radius', 0)
            requirements.append({
                'type': 'Fillet',
                'radiusInches': radius_mm / 25.4 if radius_mm else 0,
                'source': 'sw_features'
            })

        # Chamfers - convert to inches
        for chamfer in features.get('chamfers', []):
            dist_mm = chamfer.get('distance1', 0)
            requirements.append({
                'type': 'Chamfer',
                'distance1Inches': dist_mm / 25.4 if dist_mm else 0,
                'angleDegrees': chamfer.get('angle', 45),
                'source': 'sw_features'
            })

    return requirements

def compare_callout_to_requirement(callout: Dict, req: Dict, tolerance_inches: float = 0.015) -> bool:
    """Check if a drawing callout matches a SW requirement.
    Hole comparison done in INCHES with 0.015\" tolerance (~0.4mm)."""
    ctype = callout.get('calloutType')
    rtype = req.get('type')

    if ctype != rtype:
        return False

    if ctype == 'Hole':
        # Compare in INCHES
        d1 = callout.get('diameterInches', 0)
        d2 = req.get('diameterInches', 0)
        if d1 and d2 and abs(d1 - d2) <= tolerance_inches:
            return True

    elif ctype == 'TappedHole':
        # Metric threads - compare in mm
        t1 = callout.get('thread', {})
        t2 = req.get('thread', {})
        nom1 = t1.get('nominalDiameterMm', 0)
        nom2 = t2.get('nominalDiameterMm', 0)
        if nom1 and nom2 and abs(nom1 - nom2) < 0.1:
            p1 = t1.get('pitch')
            p2 = t2.get('pitch')
            if p1 and p2:
                return abs(p1 - p2) < 0.01
            return True

    elif ctype == 'Fillet':
        r1 = callout.get('radiusInches', 0)
        r2 = req.get('radiusInches', 0)
        if r1 and r2 and abs(r1 - r2) <= tolerance_inches:
            return True

    elif ctype == 'Chamfer':
        d1 = callout.get('distance1Inches', 0)
        d2 = req.get('distance1Inches', 0)
        if d1 and d2 and abs(d1 - d2) <= tolerance_inches:
            return True

    return False

def generate_diff_result(evidence: Dict, sw_data: Dict) -> Dict:
    """Compare drawing evidence against SolidWorks requirements (in inches)."""
    callouts = evidence.get('foundCallouts', [])
    requirements = extract_sw_requirements(sw_data)

    found = []
    missing = []
    matched_callouts = set()
    matched_requirements = set()

    # Check each requirement against callouts
    for ri, req in enumerate(requirements):
        match_found = False
        for ci, callout in enumerate(callouts):
            if ci not in matched_callouts and compare_callout_to_requirement(callout, req):
                found.append({
                    'status': 'FOUND',
                    'requirement': req,
                    'evidence': callout,
                    'note': f"Matched: {req.get('canonical', req.get('type'))}"
                })
                matched_callouts.add(ci)
                matched_requirements.add(ri)
                match_found = True
                break

        if not match_found:
            missing.append({
                'status': 'MISSING',
                'requirement': req,
                'evidence': None,
                'note': f"Not found in drawing: {req.get('canonical', req.get('type'))}"
            })

    # Extra callouts not matched to any requirement
    extra = []
    for ci, callout in enumerate(callouts):
        if ci not in matched_callouts:
            extra.append({
                'status': 'EXTRA',
                'requirement': None,
                'evidence': callout,
                'note': f"In drawing but not in SW: {callout.get('raw', callout.get('calloutType'))}"
            })

    diff_result = {
        'partNumber': evidence.get('partNumber'),
        'generatedAt': datetime.now().isoformat() + 'Z',
        'units': 'inches',
        'summary': {
            'totalRequirements': len(requirements),
            'found': len(found),
            'missing': len(missing),
            'extra': len(extra),
            'matchRate': f"{len(found)/len(requirements)*100:.1f}%" if requirements else "N/A"
        },
        'details': {
            'found': found,
            'missing': missing,
            'extra': extra
        }
    }

    return diff_result

# Load SW data and generate diff
if part_identity.swJsonPath:
    sw_data, err = load_json_robust(part_identity.swJsonPath)
    if sw_data:
        # Show what we're extracting
        requirements = extract_sw_requirements(sw_data)
        print("="*50)
        print("SW REQUIREMENTS EXTRACTED (inches)")
        print("="*50)
        for req in requirements:
            if req['type'] == 'Hole':
                print(f"  {req['type']}: ø{req.get('diameterInches', 0):.4f}\" ({req.get('canonical', '')})")
            else:
                print(f"  {req['type']}: {req.get('canonical', req.get('thread', {}).get('callout', ''))}")

        diff_result = generate_diff_result(evidence, sw_data)

        print("\n" + "="*50)
        print("DIFF RESULT")
        print("="*50)
        print(f"Part: {diff_result['partNumber']}")
        print(f"  Total Requirements: {diff_result['summary']['totalRequirements']}")
        print(f"  FOUND:   {diff_result['summary']['found']}")
        print(f"  MISSING: {diff_result['summary']['missing']}")
        print(f"  EXTRA:   {diff_result['summary']['extra']}")
        print(f"  Match Rate: {diff_result['summary']['matchRate']}")

        if diff_result['details']['found']:
            print("\nMatched:")
            for item in diff_result['details']['found']:
                print(f"  ✓ {item['note']}")

        if diff_result['details']['missing']:
            print("\nMissing from drawing:")
            for item in diff_result['details']['missing']:
                print(f"  ✗ {item['note']}")

        if diff_result['details']['extra']:
            print("\nExtra in drawing:")
            for item in diff_result['details']['extra']:
                print(f"  ? {item['note']}")

        # Save
        diff_out = os.path.join(OUTPUT_DIR, "DiffResult.json")
        with open(diff_out, 'w') as f:
            json.dump(diff_result, f, indent=2)
        print(f"\nSaved: {diff_out}")
    else:
        print(f"Error loading SW JSON: {err}")
else:
    print("No SW JSON path - cannot generate diff")

In [ ]:
# Cell 13: Generate QC Report with GPT-4o-mini
from openai import OpenAI
from google.colab import userdata

# Get OpenAI API key from Colab secrets
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
except:
    openai_api_key = None

if not openai_api_key:
    print("WARNING: OPENAI_API_KEY not found in Colab secrets.")
    print("Add it via: Runtime > Secrets > Add new secret")
    print("Skipping GPT-4o-mini report generation.")
else:
    client = OpenAI(api_key=openai_api_key)

    def generate_qc_report(diff_result: Dict, evidence: Dict, sw_data: Dict,
                           drawing_quality: Dict, bom_data: Dict, mfg_notes: Dict) -> str:
        """Use GPT-4o-mini to generate a comprehensive QC report."""

        # Build context for the LLM
        part_number = diff_result.get('partNumber', 'Unknown')
        summary = diff_result.get('summary', {})
        details = diff_result.get('details', {})

        # Part info
        identity = sw_data.get('identity', {})
        part_desc = identity.get('description', evidence.get('drawingInfo', {}).get('partDescription', ''))
        material = identity.get('material', evidence.get('drawingInfo', {}).get('material', ''))

        # Format found items
        found_items = []
        for item in details.get('found', []):
            req = item.get('requirement', {})
            ev = item.get('evidence', {})
            if req.get('type') == 'TappedHole':
                thread = req.get('thread', {})
                found_items.append(f"- {thread.get('callout', 'Thread')} (count: {req.get('count', 1)}) - Drawing shows: {ev.get('raw', 'N/A')}")
            elif req.get('type') == 'Hole':
                found_items.append(f"- ø{req.get('diameterInches', 0):.4f}\" {'THRU' if req.get('isThrough') else 'BLIND'} (count: {req.get('count', 1)}) - Drawing shows: {ev.get('raw', 'N/A')}")
            else:
                found_items.append(f"- {req.get('type')}: {req.get('canonical', 'N/A')} - Drawing shows: {ev.get('raw', 'N/A')}")

        # Format missing items
        missing_items = []
        for item in details.get('missing', []):
            req = item.get('requirement', {})
            if req.get('type') == 'TappedHole':
                thread = req.get('thread', {})
                missing_items.append(f"- {thread.get('callout', 'Thread')} (count: {req.get('count', 1)})")
            elif req.get('type') == 'Hole':
                missing_items.append(f"- ø{req.get('diameterInches', 0):.4f}\" {'THRU' if req.get('isThrough') else 'BLIND'} (count: {req.get('count', 1)})")
            else:
                missing_items.append(f"- {req.get('type')}: {req.get('canonical', 'N/A')}")

        # Format extra items
        extra_items = []
        for item in details.get('extra', []):
            ev = item.get('evidence', {})
            extra_items.append(f"- {ev.get('calloutType', 'Unknown')}: {ev.get('raw', 'N/A')}")

        # Drawing notes
        drawing_notes = evidence.get('drawingInfo', {}).get('notes', [])

        # === DRAWING QUALITY AUDIT DATA ===
        tb = drawing_quality.get('titleBlockCompleteness', {})
        dq = drawing_quality.get('drawingQuality', {})
        oa = drawing_quality.get('overallAssessment', {})

        # Title block checklist
        title_block_items = []
        title_block_items.append(f"- Part Number: {'✓ Present' if tb.get('hasPartNumber') else '✗ MISSING'} - {tb.get('partNumberValue', 'N/A')}")
        title_block_items.append(f"- Description: {'✓ Present' if tb.get('hasDescription') else '✗ MISSING'} - {tb.get('descriptionValue', 'N/A')[:50] if tb.get('descriptionValue') else 'N/A'}")
        title_block_items.append(f"- Material: {'✓ Present' if tb.get('hasMaterial') else '✗ MISSING'} - {tb.get('materialValue', 'N/A')}")
        title_block_items.append(f"- Revision: {'✓ Present' if tb.get('hasRevision') else '✗ MISSING'} - {tb.get('revisionValue', 'N/A')}")
        title_block_items.append(f"- Scale: {'✓ Present' if tb.get('hasScale') else '✗ MISSING'} - {tb.get('scaleValue', 'N/A')}")
        title_block_items.append(f"- Date: {'✓ Present' if tb.get('hasDate') else '✗ MISSING'} - {tb.get('dateValue', 'N/A')}")
        title_block_items.append(f"- Drawn By: {'✓ Present' if tb.get('hasDrawnBy') else '✗ MISSING'} - {tb.get('drawnByValue', 'N/A')}")
        title_block_items.append(f"- Approved By: {'✓ Present' if tb.get('hasApprovedBy') else '✗ MISSING'} - {tb.get('approvedByValue', 'N/A')}")

        # Drawing quality checklist
        quality_items = []
        quality_items.append(f"- Views Labeled: {'✓ Yes' if dq.get('viewsLabeled') else '✗ No'} - {dq.get('viewsLabeledComment', '')}")
        quality_items.append(f"- Dimensions Readable: {'✓ Yes' if dq.get('dimensionsReadable') else '✗ No'} - {dq.get('dimensionsComment', '')}")
        quality_items.append(f"- Tolerances Present: {'✓ Yes' if dq.get('tolerancesPresent') else '✗ No'} - {dq.get('tolerancesComment', '')}")
        quality_items.append(f"- Surface Finish Specified: {'✓ Yes' if dq.get('surfaceFinishSpecified') else '✗ No'} - {dq.get('surfaceFinishComment', '')}")
        quality_items.append(f"- General Tolerance Block: {'✓ Yes' if dq.get('generalToleranceBlock') else '✗ No'} - {dq.get('generalToleranceComment', '')}")
        quality_items.append(f"- Third Angle Projection Symbol: {'✓ Yes' if dq.get('thirdAngleProjection') else '✗ No'}")
        quality_items.append(f"- Units Specified: {'✓ Yes' if dq.get('unitsSpecified') else '✗ No'} - {dq.get('unitsValue', 'N/A')}")

        # === BOM DATA ===
        bom_section = ""
        if bom_data.get('hasBOM'):
            bom_items_text = []
            for item in bom_data.get('bomItems', [])[:15]:
                bom_items_text.append(f"  - Item {item.get('itemNumber', '?')}: {item.get('partNumber', 'N/A')} - {item.get('description', 'N/A')} (Qty: {item.get('quantity', 1)})")
            bom_section = f"""
### Bill of Materials (BOM):
**BOM Present:** Yes (Location: {bom_data.get('bomLocation', 'N/A')})
**Total Items:** {bom_data.get('totalItems', len(bom_data.get('bomItems', [])))}

{chr(10).join(bom_items_text) if bom_items_text else "No items extracted"}

**BOM Notes:** {bom_data.get('bomNotes', 'None')}
"""
        else:
            bom_section = """
### Bill of Materials (BOM):
**BOM Present:** No - This appears to be a part/detail drawing without a parts list.
"""

        # === MANUFACTURING NOTES ===
        ht = mfg_notes.get('heatTreatment', {})
        sf = mfg_notes.get('surfaceFinish', {})
        pc = mfg_notes.get('platingOrCoating', {})
        wn = mfg_notes.get('weldingNotes', {})
        insp = mfg_notes.get('inspectionRequirements', {})

        mfg_items = []
        mfg_items.append(f"- Heat Treatment: {'✓ ' + ht.get('specification', 'Specified') if ht.get('specified') else '✗ Not specified'}")
        mfg_items.append(f"- Surface Finish: {'✓ ' + sf.get('generalFinish', 'Specified') if sf.get('specified') else '✗ Not specified'}")
        mfg_items.append(f"- Plating/Coating: {'✓ ' + pc.get('type', 'Specified') if pc.get('specified') else '✗ Not specified'}")
        mfg_items.append(f"- Welding: {'✓ ' + wn.get('weldSpec', 'Specified') if wn.get('specified') else '✗ Not specified'}")
        mfg_items.append(f"- Inspection Requirements: {'✓ Specified' if insp.get('specified') else '✗ Not specified'}")

        general_notes = mfg_notes.get('generalNotes', [])
        special_processes = mfg_notes.get('specialProcesses', [])
        certifications = mfg_notes.get('certifications', [])

        prompt = f"""You are a Quality Control engineer reviewing an engineering drawing inspection report. You have comprehensive data from AI vision analysis including:
1. CAD-to-drawing feature comparison
2. Drawing quality/completeness audit
3. Bill of Materials extraction (if applicable)
4. Manufacturing notes and special requirements

Your job is to write a COMPREHENSIVE QC report covering ALL aspects of the drawing.

---

## PART 1: CAD FEATURE COMPARISON

**Part Number:** {part_number}
**Description:** {part_desc}
**Material (from drawing):** {material}

**CAD Model Requirements:** {summary.get('totalRequirements', 0)} features
**Drawing Match Rate:** {summary.get('matchRate', 'N/A')}

### Verified Features (Found in Drawing):
{chr(10).join(found_items) if found_items else "None"}

### Missing from Drawing (Required by CAD):
{chr(10).join(missing_items) if missing_items else "None"}

### Extra in Drawing (Not in CAD):
{chr(10).join(extra_items) if extra_items else "None"}

### Drawing Notes Observed:
{chr(10).join(f"- {note}" for note in drawing_notes[:5]) if drawing_notes else "None noted"}

---

## PART 2: DRAWING QUALITY AUDIT

### Title Block Completeness:
{chr(10).join(title_block_items)}

### Drawing Best Practices:
{chr(10).join(quality_items)}

### Overall Quality Score: {oa.get('completenessScore', 'N/A')}/10

### Issues Identified:
- Major: {', '.join(oa.get('majorIssues', [])) if oa.get('majorIssues') else 'None'}
- Minor: {', '.join(oa.get('minorIssues', [])) if oa.get('minorIssues') else 'None'}

---

## PART 3: BILL OF MATERIALS
{bom_section}

---

## PART 4: MANUFACTURING SPECIFICATIONS

### Manufacturing Requirements:
{chr(10).join(mfg_items)}

### Special Processes:
{chr(10).join(f"- {sp.get('process', 'Unknown')}: {sp.get('specification', 'N/A')}" for sp in special_processes) if special_processes else "None specified"}

### General Manufacturing Notes:
{chr(10).join(f"- {note}" for note in general_notes[:8]) if general_notes else "None"}

### Inspection Requirements:
{chr(10).join(f"- {req}" for req in insp.get('requirements', [])) if insp.get('specified') else "None specified"}

### Certifications Required:
{chr(10).join(f"- {cert}" for cert in certifications) if certifications else "None specified"}

---

## YOUR TASK

Write a professional, comprehensive QC inspection report in markdown format. Include:

1. **Executive Summary** - Overall PASS/FAIL/REVIEW NEEDED verdict with key findings
2. **Feature Verification Results** - CAD vs drawing comparison summary
3. **Drawing Quality Assessment** - Title block, views, dimensions, tolerances
4. **Bill of Materials Review** - If present, comment on completeness
5. **Manufacturing Specifications** - Heat treat, finish, coatings, special processes
6. **Issues & Action Items** - Specific problems and what needs to be fixed
7. **Confidence Level** - HIGH/MEDIUM/LOW with justification

Be specific and actionable. This report goes to manufacturing, engineering, and QC teams.
"""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a senior QC engineer who writes clear, comprehensive inspection reports. You assess feature accuracy, drawing quality, and manufacturing specifications."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=2500
        )

        return response.choices[0].message.content

    # Generate the report
    print("="*50)
    print("GENERATING QC REPORT WITH GPT-4o-mini...")
    print("="*50)

    try:
        qc_report = generate_qc_report(diff_result, evidence, sw_data, drawing_quality, bom_data, mfg_notes)

        print("\n" + qc_report)

        # Save report
        report_out = os.path.join(OUTPUT_DIR, "QCReport.md")
        with open(report_out, 'w') as f:
            f.write(f"# QC Inspection Report: {part_identity.partNumber}\n\n")
            f.write(f"**Generated:** {datetime.now().isoformat()}\n\n")
            f.write(f"**Drawing:** {DRAWING_PDF_PATH}\n\n")
            f.write("---\n\n")
            f.write(qc_report)

        print(f"\n{'='*50}")
        print(f"Saved: {report_out}")

        # Also display in Colab with formatting
        from IPython.display import display, Markdown
        display(Markdown(qc_report))

    except Exception as e:
        print(f"Error generating report: {e}")